In [15]:
!pip install yolov5 pyyaml

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 3084, in _dep_map
    return self.__dep_map
  File "/home/user/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 2877, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/requirements.py", line 35, in __init__
    parsed = _parse_requirement(requirement_string)
  File "/home/user/.local/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/_parser.py", line 64, in parse_requirement
    return _parse_re

In [16]:
!pip install pyyaml

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 3084, in _dep_map
    return self.__dep_map
  File "/home/user/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 2877, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/requirements.py", line 35, in __init__
    parsed = _parse_requirement(requirement_string)
  File "/home/user/.local/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/_parser.py", line 64, in parse_requirement
    return _parse_re

In [3]:
import yolov5
from yolov5.utils.loss import ComputeLoss




In [6]:
import xml.etree.ElementTree as ET
import yaml

from tqdm import tqdm
from yolov5.utils.general import download, Path
import torch
import tarfile

def convert_label(path, lb_path, year, image_id):
    def convert_box(size, box):
        dw, dh = 1. / size[0], 1. / size[1]
        x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
        return x * dw, y * dh, w * dw, h * dh

    in_file = open(path / f'VOC{year}/Annotations/{image_id}.xml')
    out_file = open(lb_path, 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    names = list(yaml['names'].values())  # names list
    for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls in names and int(obj.find('difficult').text) != 1:
            xmlbox = obj.find('bndbox')
            bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ('xmin', 'xmax', 'ymin', 'ymax')])
            cls_id = names.index(cls)  # class id
            out_file.write(" ".join([str(a) for a in (cls_id, *bb)]) + '\n')


# Download
dir = Path('datasets/VOC')  # dataset root dir
path = dir / 'VOCdevkit'

urls = ['http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar',  # 446MB, 5012 images
        'http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar',  # 438MB, 4953 images
        'http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar']  # 1.95GB, 17126 images
# download(urls, dir=dir / 'images', delete=False, curl=True, threads=3)
for url in urls:
    torch.hub.download_url_to_file(url, dir / 'images' / url.split('/')[-1], progress=False)

# Extract
for file in dir.glob('images/*.tar'):
    dir.mkdir(exist_ok=True, parents=True)  # create dir
    print(f'Extracting {file}...')
    tar = tarfile.open(file)
    tar.extractall(dir / 'images')
    tar.close()

# Convert
# path = dir / 'images/VOCdevkit'
for year, image_set in ('2012', 'train'), ('2012', 'val'), ('2007', 'train'), ('2007', 'val'), ('2007', 'test'):
    imgs_path = dir / 'images' / f'{image_set}{year}'
    lbs_path = dir / 'labels' / f'{image_set}{year}'
    imgs_path.mkdir(exist_ok=True, parents=True)
    lbs_path.mkdir(exist_ok=True, parents=True)

    with open(path / f'VOC{year}/ImageSets/Main/{image_set}.txt') as f:
        image_ids = f.read().strip().split()
    for id in tqdm(image_ids, desc=f'{image_set}{year}'):
        f = path / f'VOC{year}/JPEGImages/{id}.jpg'  # old img path
        lb_path = (lbs_path / f.name).with_suffix('.txt')  # new label path
        f.rename(imgs_path / f.name)  # move image
        convert_label(path, lb_path, year, id)  # convert labels to YOLO format


Extracting datasets/VOC/images/VOCtrainval_06-Nov-2007.tar...
Extracting datasets/VOC/images/VOCtest_06-Nov-2007.tar...
Extracting datasets/VOC/images/VOCtrainval_11-May-2012.tar...


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/VOC/VOCdevkit/VOC2012/ImageSets/Main/train.txt'

In [7]:
import os
from yolov5.utils.general import check_dataset

data_dict = {'path': os.path.abspath('./datasets/VOC'),
             'train': ['images/train2012', 'images/train2007', 'images/val2012', 'images/val2007'],
             'val': ['images/test2007'], 'test': ['images/test2007'],
             'names': {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat',
                       8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person',
                       15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}}

data_dict["train"] = [os.path.abspath(os.path.join(data_dict["path"], p)) for p in data_dict["train"]]
data_dict["val"] = [os.path.abspath(os.path.join(data_dict["path"], p)) for p in data_dict["val"]]

data = check_dataset(data_dict)



train_path, val_path = data["train"], data["val"]
nc, names = len(data["names"]), data["names"]

hyp = {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0,
       'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0,
       'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0,
       'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0,
       'mixup': 0.0, 'copy_paste': 0.0}

cfg = {'nc': nc, 'depth_multiple': 0.33, 'width_multiple': 0.25,
       'anchors': [[10, 13, 16, 30, 33, 23], [30, 61, 62, 45, 59, 119], [116, 90, 156, 198, 373, 326]],
       'backbone': [[-1, 1, 'Conv', [64, 6, 2, 2]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 3, 'C3', [128]],
                    [-1, 1, 'Conv', [256, 3, 2]], [-1, 6, 'C3', [256]], [-1, 1, 'Conv', [512, 3, 2]],
                    [-1, 9, 'C3', [512]], [-1, 1, 'Conv', [1024, 3, 2]], [-1, 3, 'C3', [1024]],
                    [-1, 1, 'SPPF', [1024, 5]]],
       'head': [[-1, 1, 'Conv', [512, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
                [[-1, 6], 1, 'Concat', [1]], [-1, 3, 'C3', [512, False]], [-1, 1, 'Conv', [256, 1, 1]],
                [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 4], 1, 'Concat', [1]],
                [-1, 3, 'C3', [256, False]], [-1, 1, 'Conv', [256, 3, 2]], [[-1, 14], 1, 'Concat', [1]],
                [-1, 3, 'C3', [512, False]], [-1, 1, 'Conv', [512, 3, 2]], [[-1, 10], 1, 'Concat', [1]],
                [-1, 3, 'C3', [1024, False]], [[17, 20, 23], 1, 'Detect', ['nc', 'anchors']]]}

device = "cuda"


In [20]:
from yolov5.utils.torch_utils import model_info
from yolov5.models.yolo import Model

# model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True, verbose=False)

model = Model(cfg).to(device)
model.half().float()
model.hyp = hyp # attach hyperparameters to model
model.nc = nc  # attach number of classes to model
model.names = names  # attach class names to model
model.verbose = True
model_info(model)
model


                 from  n    params  module                                  arguments                     
  0                -1  1      1760  yolov5.models.common.Conv               [3, 16, 6, 2, 2]              
  1                -1  1      4672  yolov5.models.common.Conv               [16, 32, 3, 2]                
  2                -1  1      4800  yolov5.models.common.C3                 [32, 32, 1]                   
  3                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  4                -1  2     29184  yolov5.models.common.C3                 [64, 64, 2]                   
  5                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  6                -1  3    156928  yolov5.models.common.C3                 [128, 128, 3]                 
  7                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]              
  8                -1  1    296448  

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
     

In [37]:
import numpy as np
from yolov5.utils.dataloaders import create_dataloader

train_loader, dataset = create_dataloader(
    train_path,
    640,
    1,
    max(int(model.stride.max()), 32),
    hyp=hyp,
    augment=True,
    shuffle=True,
)
labels = np.concatenate(dataset.labels, 0)
mlc = int(labels[:, 0].max())  # max label class


Scanning /tmp/notebook/datasets/VOC/labels/train2007.cache... 16551 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16551/16551 [00:00<?, ?it/s]


In [38]:
# Define the optimizer and loss function

from yolov5.utils.torch_utils import smart_optimizer
from yolov5.utils.loss import ComputeLoss

optimizer = smart_optimizer(model)
compute_loss = ComputeLoss(model)  # Define loss function

optimizer: Adam(lr=0.001) with parameter groups 57 weight(decay=0.0), 60 weight(decay=1e-05), 60 bias


In [40]:
# Train the model with tqdm

from tqdm import tqdm

num_epochs = 10

model.train()
model.to(device)

for epoch in range(num_epochs):
    pbar = tqdm(train_loader, position=0, leave=True)
    for (imgs, targets, paths, _) in pbar:
        # Forward pass
        imgs = imgs.to(device).float() / 255
        pred = model(imgs)
        loss, loss_items = compute_loss(pred, targets.to(device))
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch [{epoch}/{num_epochs}]")
        pbar.set_postfix(loss=loss.item())

Epoch [9/10]: 100%|██████████| 16551/16551 [40:49<00:00,  6.76it/s, loss=0.448]  
